Load the environment variables from a .env file

To get an api key, navigate to https://aistudio.google.com/apikey to create one. Then create a .env file in the same directory as this notebook with the following content:
```GOOGLE_API_KEY=your_api_key_here```


In [9]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=api_key)

Single shot question answering with Gemini 2.0 Flash to check correct working of the code

In [10]:
from pydantic import BaseModel


class Recipe(BaseModel):
    medicijn: str
    categorie: str
    recept: str
    medische_vraag: str
    patientinformatie: str



Create a function to get a recipe based on a medical question

In [14]:
import json
import re


def get_recipe(medical_question: str) -> Recipe:
    prompt = (
        "Je bent een medisch expert. Beantwoord de medische vraag met een recept volgens het volgende template: "
        "'Rp. [Medicijnnaam] [Sterkte] [Vorm]\ndtd. No. [Aantal]\n S. [Instructie]'. "
        "Neem relevante informatie over de patiënt in acht, zoals leeftijd en eventuele allergieën. "
        "Geef de formele medicijnnaam in het JSON-veld 'medicijn'. "
        "Benoem de categorie van het medicijn in het JSON-veld 'categorie'. "
        "Geef bij de instructie ook de duur van de behandeling aan. "
        "Herhaal de medische vraag in het JSON-veld 'medische_vraag'. "
        "Voeg de patiëntinformatie toe in het JSON-veld 'patientinformatie'. "
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=medical_question,
        config={
            "response_mime_type": "application/json",
            "response_schema": Recipe,
            "system_instruction": prompt,
        })

    try:
        parsed = extract_json_from_text(response.text)
        return Recipe.model_validate(parsed)
    except Exception as e:
        print("Fout bij het parsen van het recept:", e)
        raise


def extract_json_from_text(text: str) -> dict:
    match = re.search(r'\{.*?\}', text, re.DOTALL)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("Geen JSON gevonden in het antwoord")


question =  """ vraag: "Ik heb een keelontsteking. Heb ik antibiotica nodig?",
             "patientinformatie":
                 "leeftijd": 34,
                 "geslacht": "vrouw",
                 "koorts": True,
                 "duur_klachten_dagen": 3,
                 "pijn_bij_slikken": True,
                 "tonsillen_met_pus": True,
                 "hoesten": False,
                 "penicilline_allergie": False
             """
recipe = get_recipe(question)
print(recipe)


medicijn='Amoxicilline' categorie='Antibioticum' recept='Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. Driemaal daags 1 capsule gedurende 7 dagen.' medische_vraag='Ik heb een keelontsteking. Heb ik antibiotica nodig?' patientinformatie='leeftijd: 34, geslacht: vrouw, koorts: True, duur_klachten_dagen: 3, pijn_bij_slikken: True, tonsillen_met_pus: True, hoesten: False, penicilline_allergie: False'


In [15]:
import pandas as pd

filepath = "../data/unieke_vragen.json"
questions = pd.read_json(filepath)

questions

,vraag,patientinformatie
0,Ik heb een keelontsteking. Heb ik antibiotica ...,"{'leeftijd': 34, 'geslacht': 'vrouw', 'koorts'..."
1,Ik heb blaasontstekingklachten. Wat kan ik daa...,"{'leeftijd': 27, 'geslacht': 'vrouw', 'koorts'..."
2,Ik heb een longontsteking. Welke antibiotica z...,"{'leeftijd': 71, 'geslacht': 'man', 'koorts': ..."
3,Ik heb wondinfectieverschijnselen. Welk antibi...,"{'leeftijd': 56, 'geslacht': 'man', 'chronisch..."
4,Ik heb een tandvleesontsteking. Kan ik hier ie...,"{'leeftijd': 69, 'geslacht': 'vrouw', 'chronis..."
...,...,...
194,Ik heb slikproblemen. Zijn er medicijnen om de...,"{'leeftijd': 43, 'geslacht': 'vrouw', 'chronis..."
195,Ik heb spastische spieren door MS. Wat kan ik ...,"{'leeftijd': 43, 'geslacht': 'vrouw', 'chronis..."
196,Ik heb MS. Wat zijn mijn onderhoudsmedicijnen?,"{'leeftijd': 62, 'geslacht': 'man', 'chronisch..."
197,Ik heb een vergrote schildklier. Moet ik iets ...,"{'leeftijd': 21, 'geslacht': 'man', 'chronisch..."


In [16]:
import json
import pandas as pd

recipes = []

filepath = "../data/unieke_vragen.json"
questions = pd.read_json(filepath)

for idx, row in questions.iterrows():
    medical_context = {
        "vraag": row["vraag"],
        "patientinformatie": row["patientinformatie"],
    }
    try:
        medical_context_str = json.dumps(medical_context, ensure_ascii=False)
        recipe = get_recipe(medical_context_str)
        recipes.append(recipe)

        print(f"Recipe gegenereerd: {recipe.model_dump()}")
    except Exception as e:
        print(f"Fout bij het genereren van een recept voor deze vraag: {medical_context}")
        print(f"Error: {e}")

output_file = "../data/generated_recipes.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump([recipe.model_dump() for recipe in recipes], f, ensure_ascii=False, indent=4)

Recipe gegenereerd: {'medicijn': 'Amoxicilline', 'categorie': 'Antibioticum', 'recept': 'Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. Driemaal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Ik heb een keelontsteking. Heb ik antibiotica nodig?', 'patientinformatie': 'Patiënt is 34 jaar, vrouw, heeft koorts, klaagt 3 dagen, pijn bij slikken, tonsillen met pus, geen hoesten, geen penicilline allergie.'}
Recipe gegenereerd: {'medicijn': 'Nitrofurantoïne', 'categorie': 'Antibioticum', 'recept': 'Rp. Nitrofurantoïne 100mg capsules\ndtd. No. 10\nS. 3 maal daags 1 capsule gedurende 5 dagen.', 'medische_vraag': 'Ik heb blaasontstekingklachten. Wat kan ik daartegen nemen?', 'patientinformatie': 'Patiënt, 27 jaar, vrouw, geen koorts, wel pijn bij plassen, niet zwanger, eerder blaasontsteking gehad.'}
Recipe gegenereerd: {'medicijn': 'Amoxicilline', 'categorie': 'Antibioticum (penicilline)', 'recept': 'Rp. Amoxicilline 500mg tabletten\ndtd. No. 21\nS. Driemaal daags 1 tablet gedurende

Check each recipe for correctness. Correctness is defined as follows:
- The recipe should contain a medicine name, strength, form, number of doses, and instructions.
- The medicine name should be a valid medicine name.
- The strength should be a valid strength.
- The form should be a valid form (e.g., tablet, capsule, liquid).
- The number of doses should be a valid number.
- The instructions should include the duration of the treatment.
- The indication should be relevant to the medical question.
